# Задача

В следующей ячейке написал код по обучению модели машинного обучения для классификации Ирисов.
Необходимо реализовать веб-сервис на Flask, который бы позволял использовать эту модель для классификации через сеть.

In [1]:
from sklearn.datasets import load_iris
import pickle
from sklearn.linear_model import LogisticRegression


X, y = load_iris(return_X_y=True)
clf = LogisticRegression(random_state=0).fit(X, y)

In [2]:
with open('clf.pickle', 'wb') as f:
    f.write(pickle.dumps(clf))

In [60]:
%%writefile server.py
import pickle
import json
import numpy as np
from flask import Flask, request


app = Flask(__name__)


def load_model(pickle_path):
    with open(pickle_path, 'rb') as f:
        raw_data = f.read()
        model = pickle.loads(raw_data)
    return model

model = load_model('clf.pickle')

def classify_iris(iris_data):
    result = model.predict(np.reshape(iris_data, (1, -1)))
    return result


@app.route('/iris', methods=["GET", "POST"])
def iris_handler():
    if request.method == 'POST':
        data = request.get_json(force=True) 
        result = int(classify_iris(data['iris'])[0])
        response = {
            "result": result
        }
        return json.dumps(response)
    else:
        return "You should use only POST query"

if __name__ == '__main__':
    app.run("0.0.0.0", 8000)  # Запускаем сервер на 8000 порту

Overwriting server.py


In [61]:
! launch-server.sh server.py

Success!


In [62]:
import requests

questions = [
    [4.6, 3.1, 1.5, 0.2],
    [5.2, 2.7, 3.9, 1.4],
    [6.9, 3.1, 5.1, 2.3]
]

result = []
for q in questions:
    data = {
        'iris': q
    }

    r = requests.post("http://localhost:8000/iris", json=data)
    result.append(r.json()['result'])

После того, как вы реализуете свой веб-сервис, достаточно будет его запустить и нажать кнопку "Отправить решение". После нажатия автоматически запустится скрипт `check-server.py`, который создаст файл `result.json`. 

Сам скрипт можно использовать для проверки корректности своего решения.

In [6]:
! cat $(which check-server.py)

#!/usr/bin/env python3

import requests
import json

questions = [
    [4.6, 3.1, 1.5, 0.2],
    [5.2, 2.7, 3.9, 1.4],
    [6.9, 3.1, 5.1, 2.3]
]

result = []
for q in questions:
    data = {
        'iris': q
    }

    r = requests.post("http://localhost:8000/iris", json=data)
    result.append(r.json()['result'])

with open('/home/jovyan/work/result.json', 'w') as f:
    f.write(json.dumps(result, indent=4))